# Llama iheritune

In [1]:
def count_parameters(model):
    total_params = sum(p.numel() for p in model.parameters()) / 10**6
    print(f'total_params: {total_params:.3f}M')

In [2]:
!git clone https://github.com/EleutherAI/lm-evaluation-harness

Cloning into 'lm-evaluation-harness'...
Updating files: 100% (3320/3320), done.


In [2]:
cd lm-evaluation-harness

/home/jupyter/work/resources/lm-evaluation-harness


In [4]:
!git checkout 115206dc89dad67b8beaa90051fb52db77f0a529

Note: switching to '115206dc89dad67b8beaa90051fb52db77f0a529'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at 115206dc Merge pull request #1005 from Andrei-Aksionov/fix_indent_lm_eval_tasks_bigbench


In [3]:
!ls /home/jupyter/.local/bin

accelerate
accelerate-config
accelerate-estimate-memory
accelerate-launch
chardetect
convert-caffe2-to-onnx
convert-onnx-to-caffe2
datasets-cli
distro
f2py
get_gprof
get_objgraph
httpx
huggingface-cli
isympy
nltk
normalizer
openai
pybind11-config
pygrun
sacrebleu
torchrun
tqdm
transformers-cli
undill
virtualenv


In [4]:
import sys

for path in sys.path:
    print(path)

/home/jupyter/work/resources
/usr/lib/python310.zip
/usr/lib/python3.10
/usr/lib/python3.10/lib-dynload

/kernel/lib/python3.10/site-packages
/home/jupyter/.local/lib/python3.10/site-packages
/home/jupyter/work/resources/lm-evaluation-harness
/usr/local/lib/python3.10/dist-packages
/usr/lib/python3/dist-packages
/kernel/fallback/lib/python3.10/site-packages
/kernel/lib/python3.10/site-packages/ml_kernel/_vendor/IPython/extensions
/home/jupyter/.ipython
/kernel/lib/python3.10/site-packages/ml_kernel/_vendor/IPython/extensions


In [5]:
%pip install -e .

Defaulting to user installation because normal site-packages is not writeable
Obtaining file:///home/jupyter/work/resources/lm-evaluation-harness
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 2.5 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 1.5 MB/s eta 0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 7.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━

In [5]:
pwd

'/home/jupyter/work/resources/lm-evaluation-harness'

In [6]:
import os

# Получаем текущий PATH
current_path = os.environ.get('PATH', '')

# Директория, которую нужно добавить
new_directory = '/home/jupyter/.local/bin'

# Проверяем, есть ли уже этот путь в PATH, чтобы избежать дублирования
if new_directory not in current_path:
    # Добавляем новый путь в PATH
    os.environ['PATH'] = f"{new_directory}:{current_path}"

# Проверяем, что путь добавлен
print(os.environ['PATH'])

/home/jupyter/.local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin


In [7]:
import os

print(os.environ['PATH'])

/home/jupyter/.local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin


In [ ]:
# !pip install evaluate

In [8]:
import sacrebleu
print(sacrebleu.__version__)

1.5.0


In [161]:
import gc
import os
import json
import sys
from pathlib import Path
from typing import Dict, List, Literal, Optional

from lm_eval import evaluator, tasks

import torch
import numpy as np
from tqdm.auto import tqdm
from torch.nn import Linear, CrossEntropyLoss
from datasets import load_dataset, concatenate_datasets
from accelerate import DataLoaderConfiguration


from transformers import (
    BertTokenizer,
    BertModel, 
    default_data_collator, 
    pipeline, 
    AutoModelForSequenceClassification,
    LlamaForCausalLM,
    LlamaTokenizer,
    AutoConfig,
    AdamW,
    Trainer, 
    TrainingArguments,
    DataCollatorForLanguageModeling,
    AutoModelForCausalLM,
    AutoTokenizer
)

from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

from lm_eval.base import BaseLM

In [10]:
parent_llama = LlamaForCausalLM.from_pretrained("openlm-research/open_llama_3b_v2")

count_parameters(parent_llama)

total_params: 3426.474M


In [11]:
def filter_dataset(dataset, percents):
    part = percents / 100
    return dataset.shuffle(seed=42).select(range(int(len(dataset)*part)))

In [12]:
wikipedia_dataset = load_dataset('wikipedia', '20220301.en', split='train[:1%]')
bookcorpus_dataset = load_dataset('bookcorpus', split='train[:1%]')

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for wikipedia contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wikipedia
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [13]:
print('upd')

upd


In [42]:
wikipedia_dataset_filtered = filter_dataset(wikipedia_dataset, 10)
bookcorpus_dataset_filtered = filter_dataset(bookcorpus_dataset, 10)

In [43]:
print(len(wikipedia_dataset_filtered))
print(len(bookcorpus_dataset_filtered))

6458
74004


In [44]:
parent_config = AutoConfig.from_pretrained("openlm-research/open_llama_3b_v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [45]:
print(parent_config)

LlamaConfig {
  "_name_or_path": "openlm-research/open_llama_3b_v2",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 3200,
  "initializer_range": 0.02,
  "intermediate_size": 8640,
  "max_position_embeddings": 2048,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 26,
  "num_key_value_heads": 32,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.41.2",
  "use_cache": true,
  "vocab_size": 32000
}



In [72]:
child_config = AutoConfig.from_pretrained("openlm-research/open_llama_3b_v2", num_hidden_layers=6)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [73]:
print(child_config)

LlamaConfig {
  "_name_or_path": "openlm-research/open_llama_3b_v2",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 3200,
  "initializer_range": 0.02,
  "intermediate_size": 8640,
  "max_position_embeddings": 2048,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 6,
  "num_key_value_heads": 32,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.41.2",
  "use_cache": true,
  "vocab_size": 32000
}



In [74]:
## take 1 2 4 6 8 10 12 14 16 18 20 22 24 26 layers of parent model
layers = [0, 3, 10, 15, 22, 25]
print(len(layers))

6


In [170]:
child_llama = LlamaForCausalLM(child_config)

for i in range(6):
    child_llama.model.layers[i].load_state_dict(parent_llama.model.layers[layers[i]].state_dict())
    
print(count_parameters(child_llama))

total_params: 948.266M
None


In [36]:
%pip install sentencepiece

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 5.8 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [131]:
import sentencepiece
print(sentencepiece.__version__)

0.2.0


In [132]:
tokenizer=LlamaTokenizer.from_pretrained(
    "openlm-research/open_llama_3b_v2",
#     pad_token="<pad>",
)
tokenizer.pad_token_id = 0

In [133]:
tokenizer

LlamaTokenizer(name_or_path='openlm-research/open_llama_3b_v2', vocab_size=32000, model_max_length=2048, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}

In [134]:
def tokenize_function(examples):
    texts = [text for text in examples['text'] if text is not None]
    return tokenizer(
        texts,
        return_special_tokens_mask=True,
        truncation=True,
        max_length=2048,
        padding='max_length'
    )

In [135]:
tokenized_wikipedia_dataset = wikipedia_dataset_filtered.map(
    tokenize_function,
    batched=True,
    remove_columns=["text"]
)

In [136]:
tokenized_bookcorpus_dataset = bookcorpus_dataset_filtered.map(
    tokenize_function,
    batched=True,
    remove_columns=["text"]
)

In [137]:
combined_dataset = concatenate_datasets([tokenized_wikipedia_dataset, tokenized_bookcorpus_dataset])

In [138]:
print("upd")

upd


In [139]:
len(combined_dataset)

80462

In [140]:
BATCH_SIZE = 8
NUM_EPOCHS = 6
PER_DEVICE_TRAIN_BATCH_SIZE = 8

In [141]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

In [142]:
training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=PER_DEVICE_TRAIN_BATCH_SIZE,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
    fp16=True,
    gradient_accumulation_steps=8,
    report_to=None,
    dataloader_num_workers=4
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [157]:
# child_llama.to('cpu')
# del child_llama
# del trainer
gc.collect()
torch.cuda.empty_cache()
for device in range(torch.cuda.device_count()):
    torch.cuda.set_device(device)
    torch.cuda.empty_cache()
gc.collect()

0

In [143]:
class CustomDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        return item

In [144]:
train_dataset = CustomDataset(combined_dataset)

In [145]:
trainer = Trainer(
    model=child_llama,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

In [146]:
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
trainer.train()

  7%|▋         | 500/7542 [48:10<11:18:26,  5.78s/it]

{'loss': 5.604, 'grad_norm': 2.7871007919311523, 'learning_rate': 4.668522938212676e-05, 'epoch': 0.4}


 13%|█▎        | 1000/7542 [1:36:22<10:31:14,  5.79s/it]

{'loss': 4.5772, 'grad_norm': 3.0762715339660645, 'learning_rate': 4.3370458764253514e-05, 'epoch': 0.8}


 20%|█▉        | 1500/7542 [2:24:38<9:43:11,  5.79s/it] 

...

 99%|█████████▉| 7500/7542 [12:29:39<04:03,  5.80s/it]

{'loss': 2.4324, 'grad_norm': 4.2724528312683105, 'learning_rate': 2.9169981437284544e-07, 'epoch': 5.97}


100%|██████████| 7542/7542 [12:33:44<00:00,  6.00s/it]

{'train_runtime': 45224.0799, 'train_samples_per_second': 10.675, 'train_steps_per_second': 0.167, 'train_loss': 3.4635748993462134, 'epoch': 6.0}


TrainOutput(global_step=7542, training_loss=3.4635748993462134, metrics={'train_runtime': 45224.0799, 'train_samples_per_second': 10.675, 'train_steps_per_second': 0.167, 'total_flos': 5.016953439151718e+18, 'train_loss': 3.4635748993462134, 'epoch': 5.998806919864784})

In [66]:
print("upd")

upd


In [ ]:
print("upd2")

In [ ]:


# Сохранение дообученной модели
trainer.save_model("./open_llama_3b_v2_first_last_step")
tokenizer.save_pretrained("./open_llama_3b_v2_first_last_step")

('./open_llama_3b_v2_first_last_step/tokenizer_config.json',
 './open_llama_3b_v2_first_last_step/special_tokens_map.json',
 './open_llama_3b_v2_first_last_step/tokenizer.model',
 './open_llama_3b_v2_first_last_step/added_tokens.json')

In [ ]:
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

child_llama.to('cpu')
gc.collect()
torch.cuda.empty_cache()
for device in range(torch.cuda.device_count()):
    torch.cuda.set_device(device)
    torch.cuda.empty_cache()
gc.collect()

In [158]:
BATCH_SIZE = 8
PER_DEVICE_TRAIN_BATCH_SIZE = BATCH_SIZE 
NUM_EPOCHS = 3
OUTPUT_DIR = "./output"

In [31]:
print("upd1")

upd1


Наследуем student model

In [159]:
class ModelWrapper(BaseLM):
    def __init__(
        self,
        model,
        batch_size,
        tokenizer,
        device
    ):
        super().__init__()
        self.config = model.config
        self.model = model
        if torch.cuda.device_count() > 1:
            self.model = torch.nn.DataParallel(self.model)
        self.model.to(device)
        self.tokenizer = tokenizer
        self.batch_size_per_gpu = batch_size
        self.device_ = device

    @torch.inference_mode()
    def _model_call(self, inps):
        outputs = self.model(inps)
        if hasattr(outputs, 'logits'):
            return outputs.logits
        elif hasattr(outputs, 'last_hidden_state'):
            return outputs.last_hidden_state
        else:
            raise ValueError("Model output does not contain 'logits' or 'last_hidden_state'")

    @torch.inference_mode()
    def _model_generate(self, context, max_length, eos_token_id) -> torch.Tensor:
        # this only supports batch size 1
        assert context.shape[0] == 1
        out = generate(self.model, context[0], max_length, eos_id=eos_token_id)
        for block in self.model.transformer.h:
            block.attn.kv_cache.reset_parameters()
        return out.unsqueeze(0)

    @property
    def batch_size(self):
        return self.batch_size_per_gpu*torch.cuda.device_count()

    @property
    def device(self):
        return self.device_

    @property
    def eot_token_id(self):
        # we use EOT because end of *text* is more accurate for what we're doing than end of *sentence*
        return self.tokenizer.eos_id

    @property
    def max_gen_toks(self):
        return 256

    @property
    def max_length(self):
        return self.config.max_position_embeddings

    def tok_encode(self, string: str) -> List[int]:
        return self.tokenizer.encode(string)

    def tok_decode(self, tokens: List[int]) -> str:
        t = torch.tensor(tokens)
        return self.tokenizer.decode(t)
    
    def clear_gpu_memory(self):
        self.model.module.cpu()
        del self.model  
        gc.collect()
        torch.cuda.empty_cache()
        for device in range(torch.cuda.device_count()):
            torch.cuda.set_device(device)
            torch.cuda.empty_cache()
        gc.collect()

In [163]:
model_path = "./open_llama_3b_v2_first_last_step"
model = LlamaForCausalLM.from_pretrained(model_path)

tokenizer=LlamaTokenizer.from_pretrained(
    model_path,
#     pad_token="<pad>",
)

In [164]:
eval_tasks: List[str] = ['winogrande','boolq','piqa']

num_fewshot = 0
limit = 256
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

wrapped_model = ModelWrapper(
    model=model,
    batch_size=BATCH_SIZE,
    tokenizer=tokenizer,
    device=device
)

In [165]:
results = evaluator.evaluate(
    lm=wrapped_model,
    task_dict=tasks.get_task_dict(eval_tasks),
    num_fewshot=num_fewshot
)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for winogrande contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/winogrande
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for super_glue contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/super_glue
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/data

Task: winogrande; number of docs: 1267
Task: winogrande; document 0; context prompt (starting on next line):
People think Rebecca
(end of prompt on previous line)
Requests: [Req_loglikelihood('People think Samantha', ' is embarassed, because Samantha made snide comments about the shirt Rebecca was wearing.')[0]
, Req_loglikelihood('People think Rebecca', ' is embarassed, because Samantha made snide comments about the shirt Rebecca was wearing.')[0]
]
Task: boolq; number of docs: 3270
Task: boolq; document 0; context prompt (starting on next line):
NCIS: New Orleans (season 4) -- The fourth season of NCIS: New Orleans premiered on September 26, 2017 on CBS. The series continues to air following Bull, Tuesday at 10:00 p.m. (ET) and contained 24 episodes. The season concluded on May 15, 2018.
Question: is ncis new orleans over for the season?
Answer:
(end of prompt on previous line)
Requests: (Req_loglikelihood('NCIS: New Orleans (season 4) -- The fourth season of NCIS: New Orleans premie

100%|██████████| 12750/12750 [03:12<00:00, 66.08it/s] 


In [ ]:
# wrapped_model.clear_gpu_memory()

In [166]:
print(results)

{'results': {'winogrande': {'acc': 0.5027624309392266, 'acc_stderr': 0.014052271211616438}, 'boolq': {'acc': 0.37920489296636084, 'acc_stderr': 0.008486012137246294}, 'piqa': {'acc': 0.5201305767138193, 'acc_stderr': 0.011656365410780368, 'acc_norm': 0.5059847660500544, 'acc_norm_stderr': 0.011664988455853328}}, 'versions': {'winogrande': 0, 'boolq': 1, 'piqa': 0}}


In [167]:
eval_tasks: List[str] = ['winogrande','boolq','piqa']

num_fewshot = 0
limit = 256
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

wrapped_model = ModelWrapper(
    model=parent_llama,
    batch_size=BATCH_SIZE,
    tokenizer=tokenizer,
    device=device
)

In [ ]:
results_parent = evaluator.evaluate(
    lm=wrapped_model,
    task_dict=tasks.get_task_dict(eval_tasks),
    num_fewshot=num_fewshot
)

Using the latest cached version of the module from /tmp/xdg_cache/huggingface/modules/datasets_modules/datasets/winogrande/a826c3d3506aefe0e9e9390dcb53271070536586bab95849876b2c1743df56e2 (last modified on Sun Jun  2 10:50:29 2024) since it couldn't be found locally at winogrande, or remotely on the Hugging Face Hub.


Task: winogrande; number of docs: 1267
Task: winogrande; document 0; context prompt (starting on next line):
People think Rebecca
(end of prompt on previous line)
Requests: [Req_loglikelihood('People think Samantha', ' is embarassed, because Samantha made snide comments about the shirt Rebecca was wearing.')[0]
, Req_loglikelihood('People think Rebecca', ' is embarassed, because Samantha made snide comments about the shirt Rebecca was wearing.')[0]
]
Task: boolq; number of docs: 3270
Task: boolq; document 0; context prompt (starting on next line):
NCIS: New Orleans (season 4) -- The fourth season of NCIS: New Orleans premiered on September 26, 2017 on CBS. The series continues to air following Bull, Tuesday at 10:00 p.m. (ET) and contained 24 episodes. The season concluded on May 15, 2018.
Question: is ncis new orleans over for the season?
Answer:
(end of prompt on previous line)
Requests: (Req_loglikelihood('NCIS: New Orleans (season 4) -- The fourth season of NCIS: New Orleans premie

100%|██████████| 12750/12750 [09:37<00:00, 22.08it/s] 


In [ ]:
print(results_parent)

{'results': {'winogrande': {'acc': 0.6566692975532754, 'acc_stderr': 0.013344823185358012}, 'boolq': {'acc': 0.6672782874617736, 'acc_stderr': 0.00824112701829168}, 'piqa': {'acc': 0.7676822633297062, 'acc_stderr': 0.009853201384168243, 'acc_norm': 0.7763873775843307, 'acc_norm_stderr': 0.009721489519176283}}, 'versions': {'winogrande': 0, 'boolq': 1, 'piqa': 0}}


In [ ]:
print("upd2")

In [171]:
eval_tasks: List[str] = ['winogrande','boolq','piqa']

num_fewshot = 0
limit = 256
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

wrapped_model = ModelWrapper(
    model=child_llama,
    batch_size=BATCH_SIZE,
    tokenizer=tokenizer,
    device=device
)

In [172]:
results_child = evaluator.evaluate(
    lm=wrapped_model,
    task_dict=tasks.get_task_dict(eval_tasks),
    num_fewshot=num_fewshot
)

Using the latest cached version of the module from /tmp/xdg_cache/huggingface/modules/datasets_modules/datasets/winogrande/a826c3d3506aefe0e9e9390dcb53271070536586bab95849876b2c1743df56e2 (last modified on Sun Jun  2 10:50:29 2024) since it couldn't be found locally at winogrande, or remotely on the Hugging Face Hub.


Task: winogrande; number of docs: 1267
Task: winogrande; document 0; context prompt (starting on next line):
People think Rebecca
(end of prompt on previous line)
Requests: [Req_loglikelihood('People think Samantha', ' is embarassed, because Samantha made snide comments about the shirt Rebecca was wearing.')[0]
, Req_loglikelihood('People think Rebecca', ' is embarassed, because Samantha made snide comments about the shirt Rebecca was wearing.')[0]
]
Task: boolq; number of docs: 3270
Task: boolq; document 0; context prompt (starting on next line):
NCIS: New Orleans (season 4) -- The fourth season of NCIS: New Orleans premiered on September 26, 2017 on CBS. The series continues to air following Bull, Tuesday at 10:00 p.m. (ET) and contained 24 episodes. The season concluded on May 15, 2018.
Question: is ncis new orleans over for the season?
Answer:
(end of prompt on previous line)
Requests: (Req_loglikelihood('NCIS: New Orleans (season 4) -- The fourth season of NCIS: New Orleans premie

100%|██████████| 12750/12750 [03:16<00:00, 65.05it/s] 


In [173]:
print(results_child)

{'results': {'winogrande': {'acc': 0.4996053670086819, 'acc_stderr': 0.014052481306049516}, 'boolq': {'acc': 0.5134556574923548, 'acc_stderr': 0.00874188773591556}, 'piqa': {'acc': 0.5310119695321001, 'acc_stderr': 0.011643363511107454, 'acc_norm': 0.5255712731229597, 'acc_norm_stderr': 0.011650557844573573}}, 'versions': {'winogrande': 0, 'boolq': 1, 'piqa': 0}}


Выводы

Метод Inheritune:

Преимущества: Быстрое обучение, меньше данных для обучения.

Недостатки: Ограниченная гибкость архитектуры, зависит от качества и объема данных.

Метод дистилляции знаний:

Преимущества: Высокая точность, особенно при наличии большого объема данных.

Недостатки: Высокие вычислительные затраты, сложность настройки.

Когда использовать:

Inheritune: Подходит для случаев с ограниченными вычислительными ресурсами и данными. Идеально для быстрого прототипирования и ситуаций, когда необходима производительность на устройствах с ограниченными ресурсами.
Дистилляция знаний: Подходит для случаев, когда требуется высокая точность и есть доступ к большому объему данных и вычислительных ресурсов. Идеально для конечных моделей в производственных условиях.